In [4]:
!pip install -q langchain chromadb bs4 langchain-community
!pip install -U langchain-ollama

In [8]:
!pip install -q langchain PyMuPDF

In [6]:
nome_arquivo = "PGDE0269-D.pdf"

In [9]:
from langchain.document_loaders import PyMuPDFLoader

# nome_arquivo = list(uploaded.keys())[0]  # Pega o primeiro arquivo enviado

loader = PyMuPDFLoader(nome_arquivo)
documents = loader.load()

In [12]:
# Dividir o conteúdo em partes (chunks)
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(documents)

print(f"Chunks gerados: {len(docs)}")

Chunks gerados: 290


In [16]:
!pip install -q tqdm

In [24]:
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import Chroma
from tqdm import tqdm

# Inicializa os embeddings
embeddings = OllamaEmbeddings(model="llama2")

# Cria vetor vazio do Chroma, ou seja sem documentos
vectorstore = Chroma(embedding_function=embeddings, persist_directory="chroma_db")

# Adiciona os documentos
for doc in tqdm(docs, desc="Gerando embeddings"):
    vectorstore.add_documents([doc])

Gerando embeddings: 100%|██████████| 290/290 [3:51:39<00:00, 47.93s/it]  


In [27]:
from langchain.llms import Ollama

llm = Ollama(model="llama2")

C:\Users\wende\AppData\Local\Temp\ipykernel_6392\3081704472.py:3: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama2")


In [31]:
from langchain.chains import RetrievalQA

# Criar o pipeline de RAG
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever()
)

In [36]:
pergunta = "Quais são os principais desafios da inclusão de pessoas com deficiência em universidades brasileiras? Envie a resposta em português brasil."

In [39]:
resposta_sem_rag = llm(pergunta)
print("Resposta sem RAG:", resposta_sem_rag)

C:\Users\wende\AppData\Local\Temp\ipykernel_6392\912322053.py:1: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  resposta_sem_rag = llm(pergunta)


Resposta sem RAG: 
 Os principais desafios para a inclusão de pessoas com deficiência em universidades brasileiras incluem:

1. Acessibilidade física: Many universities in Brazil lack wheelchair ramps, elevators, and other accessibility features, making it difficult or impossible for people with mobility impairments to navigate the campus.
2. Lack of accommodations: Universities may not have the necessary accommodations to support students with disabilities, such as adapted classrooms, assistive technology, or sign language interpreters.
3. Social stigma: People with disabilities may face social stigma and discrimination from their peers, which can create a hostile environment for them on campus.
4. Limited resources: Universities may not have the necessary resources to provide adequate support services for students with disabilities, such as counseling or academic support.
5. Lack of representation: People with disabilities may be underrepresented in university administrations and dec

In [43]:
# Pergunta utilizando o pipeline RAG
resposta_com_rag = qa_chain.run(pergunta)
print("Resposta com RAG:", resposta_com_rag)

C:\Users\wende\AppData\Local\Temp\ipykernel_6392\1019417238.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  resposta_com_rag = qa_chain.run(pergunta)


Resposta com RAG: Based on the provided context, the main challenges faced by people with disabilities in Brazilian universities are:

1. Lack of accessibility: Many universities in Brazil still lack proper infrastructure and facilities to accommodate students with disabilities, such as wheelchair ramps, elevators, and adapted classrooms.
2. Limited availability of reservation of quotas: The system of reservation of quotas for people with disabilities is not yet fully implemented in all universities, which can limit the access of these individuals to higher education.
3. Social stigma: People with disabilities may face social stigma and discrimination, which can affect their self-esteem and academic performance.
4. Limited financial resources: Many students with disabilities may struggle to afford the costs of higher education due to insufficient financial support from the government or their families.
5. Lack of accommodation: Some universities may not have adapted accommodations for 

In [51]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

template = """
Você é um especialista em Design Universal, acessibilidade educacional ou inclusão digita e irá responder a perguntas com base na Gestão de Design aplicada à Inclusão e Acessibilidade de Estudantes com  Deficiência no Ensino Superior.

Utilize **somente o conteúdo dos documentos a seguir** para elaborar a resposta. Seja direto, objetivo e inclua o artigo específico, se citado. 
Envie resposta em Português PT-BR
Formule uma resposta em até 20 palavras

{context}

Pergunta: {question}
Resposta:
"""

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=template,
)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorstore.as_retriever(),
    chain_type="stuff",
    chain_type_kwargs={"prompt": prompt},
)

resposta = qa_chain.invoke({"query": pergunta})

print("Pergunta do Usuário:")
print(resposta["query"])
print("\nResposta do Bot:")
print(resposta["result"])

Pergunta do Usuário:
Quais são os principais desafios da inclusão de pessoas com deficiência em universidades brasileiras? Envie a resposta em português brasil.

Resposta do Bot:
Os principais desafios da inclusão de pessoas com deficiência em universidades brasileiras incluem:

1. Acessibilidade física e ambiental: As instituições de ensino superior podem precisar realizar melhorias na acessibilidade física e ambiental, como o uso de rampas, elevadores, áreas de reunião acessíveis, entre outras medidas.
2. Adaptação do currículo: É importante que o currículo seja adaptado para atender às necessidades dos estudantes com deficiência, incluindo a utilização de recursos tecnológicos e didática innovadora.
3. Acompanimento e suporte: As universidades precisam oferecer apoio e acompanhamento especializado aos estudantes com deficiência, para garantir o sucesso na navegação acadêmica.
4. Lack of resources: Many Brazilian universities face resource constraints, which can hinder the implementa

--------------------------------------------
INTELIGÊNCIA ARTIFICIAL
NOME: Wendel Victor Santos Xavier
RA: 972310140
I͏nt͏elig͏ênci͏a artif͏icial ͏com ͏RAG (͏Resgat͏e-Aume͏ntado Ge͏ração) pr͏o͏cura inf͏ormações em papéis f͏or͏a (͏como P͏DFs) antes de͏ respon͏de͏r- enquanto ͏a ͏inteli͏gência artificial s͏em RAG respond͏e só ͏com base no que já foi ensinada. Se͏m usar ͏o RAG- eu vi isso agora que a resposta fi͏ca mais básica e͏ com ͏uma ͏tendên͏ci͏a a faz͏er imagens erradas.͏ Já com͏ ͏o RAG a resposta fica mais completa porqu͏e le͏va e͏m conta o conteú͏do do documento ͏inserido- e quando͏ se usa RAG junto com͏ um comando bem organizado as resposta se torna mais aprofundada com ma͏ior base e con͏texto. Usando Ollama com ͏Ju͏pyterLab dá͏ para fazer e͏sse proces͏so localmente͏ misturando IA e dados sob medida.